In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Sjögrens_Syndrome"
cohort = "GSE51092"

# Input paths
in_trait_dir = "../../input/GEO/Sjögrens_Syndrome"
in_cohort_dir = "../../input/GEO/Sjögrens_Syndrome/GSE51092"

# Output paths
out_data_file = "../../output/preprocess/Sjögrens_Syndrome/GSE51092.csv"
out_gene_data_file = "../../output/preprocess/Sjögrens_Syndrome/gene_data/GSE51092.csv"
out_clinical_data_file = "../../output/preprocess/Sjögrens_Syndrome/clinical_data/GSE51092.csv"
json_path = "../../output/preprocess/Sjögrens_Syndrome/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Variants at multiple loci implicated in both innate and adaptive immune responses are associated with Sjögren’s syndrome"
!Series_summary	"This is a genome-wide association study performed in Sjogrens syndrome in which gene expression data was used in conjunction with genotype data to perform expression quantitative trait loci (eQTL) analysis."
!Series_overall_design	"This is a case/control study."
Sample Characteristics Dictionary:
{0: ['disease state: none', 'disease state: Sjögrens syndrome']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information and series title, this study involves gene expression data
# alongside genotype data for eQTL analysis, so this dataset likely contains gene expression data.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Looking at the sample characteristics dictionary, there's only one key (0) with disease state information
trait_row = 0  # The disease state is recorded in key 0
age_row = None  # No age information is available in the sample characteristics
gender_row = None  # No gender information is available in the sample characteristics

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trait values to binary format (0 for control, 1 for case)"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary based on presence of Sjögren's syndrome
    if 'none' in value.lower():
        return 0  # Control
    elif 'sjogren' in value.lower() or 'sjögren' in value.lower() or 'sjogrens' in value.lower():
        return 1  # Case with Sjögren's syndrome
    else:
        return None  # Unknown or undefined

# No age data available, but define the conversion function for completeness
def convert_age(value):
    """Convert age values to continuous format"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

# No gender data available, but define the conversion function for completeness
def convert_gender(value):
    """Convert gender values to binary format (0 for female, 1 for male)"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    else:
        return None

# 3. Save Metadata
# Trait data is available since trait_row is not None
is_trait_available = trait_row is not None
validate_and_save_cohort_info(is_final=False, 
                              cohort=cohort, 
                              info_path=json_path, 
                              is_gene_available=is_gene_available, 
                              is_trait_available=is_trait_available)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we proceed with clinical feature extraction
if trait_row is not None:
    # Get clinical features using the provided function
    clinical_selected = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    # Preview the selected clinical features
    preview = preview_df(clinical_selected)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the clinical data to a CSV file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_selected.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'GSM1238429': [0.0], 'GSM1238430': [0.0], 'GSM1238431': [0.0], 'GSM1238432': [0.0], 'GSM1238433': [0.0], 'GSM1238434': [0.0], 'GSM1238435': [0.0], 'GSM1238436': [0.0], 'GSM1238437': [0.0], 'GSM1238438': [0.0], 'GSM1238439': [0.0], 'GSM1238440': [0.0], 'GSM1238441': [0.0], 'GSM1238442': [0.0], 'GSM1238443': [0.0], 'GSM1238444': [0.0], 'GSM1238445': [0.0], 'GSM1238446': [0.0], 'GSM1238447': [0.0], 'GSM1238448': [0.0], 'GSM1238449': [0.0], 'GSM1238450': [0.0], 'GSM1238451': [0.0], 'GSM1238452': [0.0], 'GSM1238453': [0.0], 'GSM1238454': [0.0], 'GSM1238455': [0.0], 'GSM1238456': [0.0], 'GSM1238457': [0.0], 'GSM1238458': [0.0], 'GSM1238459': [0.0], 'GSM1238460': [0.0], 'GSM1238461': [1.0], 'GSM1238462': [1.0], 'GSM1238463': [1.0], 'GSM1238464': [1.0], 'GSM1238465': [1.0], 'GSM1238466': [1.0], 'GSM1238467': [1.0], 'GSM1238468': [1.0], 'GSM1238469': [1.0], 'GSM1238470': [1.0], 'GSM1238471': [1.0], 'GSM1238472': [1.0], 'GSM1238473': [1.0], 'GSM1238474': [

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651232', 'ILMN_1651254', 'ILMN_1651262', 'ILMN_1651278',
       'ILMN_1651282', 'ILMN_1651315', 'ILMN_1651316', 'ILMN_1651336',
       'ILMN_1651341', 'ILMN_1651346', 'ILMN_1651347', 'ILMN_1651354',
       'ILMN_1651373', 'ILMN_1651378', 'ILMN_1651385', 'ILMN_1651403'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers (ILMN_*) are Illumina microarray probe IDs, not human gene symbols.
# They are used on Illumina microarray platforms and need to be mapped to human gene symbols.
# The "ILMN_" prefix is specific to Illumina's BeadArray technology.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1825594', 'ILMN_1810803', 'ILMN_1722532', 'ILMN_1884413', 'ILMN_1906034'], 'Species': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Source': ['Unigene', 'RefSeq', 'RefSeq', 'Unigene', 'Unigene'], 'Search_Key': ['ILMN_89282', 'ILMN_35826', 'ILMN_25544', 'ILMN_132331', 'ILMN_105017'], 'Transcript': ['ILMN_89282', 'ILMN_35826', 'ILMN_25544', 'ILMN_132331', 'ILMN_105017'], 'ILMN_Gene': ['HS.388528', 'LOC441782', 'JMJD1A', 'HS.580150', 'HS.540210'], 'Source_Reference_ID': ['Hs.388528', 'XM_497527.2', 'NM_018433.3', 'Hs.580150', 'Hs.540210'], 'RefSeq_ID': [nan, 'XM_497527.2', 'NM_018433.3', nan, nan], 'Unigene_ID': ['Hs.388528', nan, nan, 'Hs.580150', 'Hs.540210'], 'Entrez_Gene_ID': [nan, 441782.0, 55818.0, nan, nan], 'GI': [23525203.0, 89042416.0, 46358420.0, 7376124.0, 5437312.0], 'Accession': ['BU678343', 'XM_497527.2', 'NM_018433.3', 'AW629334', 'AI818233'], 'Symbol': [nan, 'LOC441782', 'JMJD1A', nan, nan], 'Prot

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns contain the probe IDs and gene symbols
# Looking at the previewed data, we see 'ID' contains ILMN_* identifiers matching our gene expression data
# 'Symbol' column has gene symbols, although some values are NaN

# 2. Get the gene mapping dataframe using the identified columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')

# Print a preview of the mapping to verify
print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
# The function already handles the equal distribution of expression values among multiple genes
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Normalize gene symbols to standard format
gene_data = normalize_gene_symbols_in_index(gene_data)

# Print the first few rows of the resulting gene expression data
print("\nGene expression data preview (first 5 genes):")
print(gene_data.head())


Gene mapping preview:
{'ID': ['ILMN_1810803', 'ILMN_1722532', 'ILMN_1708805', 'ILMN_1672526', 'ILMN_2185604'], 'Gene': ['LOC441782', 'JMJD1A', 'NCOA3', 'LOC389834', 'C17orf77']}



Gene expression data preview (first 5 genes):
       GSM1238429  GSM1238430  GSM1238431  GSM1238432  GSM1238433  GSM1238434  \
Gene                                                                            
AAAS     5.032245    6.004850    5.591386    5.376855    5.559783    5.522240   
AACS     5.024746    5.294624    5.537792    4.939043    5.455228    5.237456   
AAMDC    5.134898    5.628010    5.274912    5.831238    3.939284    5.555536   
AAMP     5.796785    6.223586    6.184308    6.635248    5.909207    6.629351   
AAR2     8.115394    8.362359    8.134386    7.866097    8.300355    7.922875   

       GSM1238435  GSM1238436  GSM1238437  GSM1238438  ...  GSM1238647  \
Gene                                                   ...               
AAAS     5.372631    5.275120    5.577633    5.642134  ...    5.661638   
AACS     4.428036    5.213866    5.271462    4.650578  ...    4.487466   
AAMDC    6.733012    6.498110    6.253775    5.790463  ...    5.044327   
AAMP     6.5767

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the previously saved clinical data
clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
print(f"Loaded clinical data shape: {clinical_df.shape}")
print(clinical_df.head())

# 3. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print(linked_data.head())

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape after handling missing values: {linked_data.shape}")

# 5. Determine whether the trait and demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=f"Dataset contains gene expression data from CD4 T-cells of pSS patients and healthy controls."
)

# 7. Save the data if it's usable
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset is not suitable for association studies.")

Normalized gene data shape: (10038, 222)
First few normalized gene symbols: ['AAAS', 'AACS', 'AAMDC', 'AAMP', 'AAR2', 'AARS1', 'AARS2', 'AARSD1', 'AASDH', 'AASDHPPT']


Normalized gene data saved to ../../output/preprocess/Sjögrens_Syndrome/gene_data/GSE51092.csv
Loaded clinical data shape: (1, 222)
                   GSM1238429  GSM1238430  GSM1238431  GSM1238432  GSM1238433  \
Sjögrens_Syndrome         0.0         0.0         0.0         0.0         0.0   

                   GSM1238434  GSM1238435  GSM1238436  GSM1238437  GSM1238438  \
Sjögrens_Syndrome         0.0         0.0         0.0         0.0         0.0   

                   ...  GSM1238647  GSM1238648  GSM1238649  GSM1238650  \
Sjögrens_Syndrome  ...         1.0         1.0         1.0         1.0   

                   GSM1238651  GSM1238652  GSM1238653  GSM1238654  GSM1238655  \
Sjögrens_Syndrome         1.0         1.0         1.0         1.0         1.0   

                   GSM1238656  
Sjögrens_Syndrome         1.0  

[1 rows x 222 columns]
Linked data shape: (222, 10039)
            Sjögrens_Syndrome      AAAS      AACS     AAMDC      AAMP  \
GSM1238429                0.0  5.0322

Shape after handling missing values: (222, 10039)
For the feature 'Sjögrens_Syndrome', the least common label is '0.0' with 32 occurrences. This represents 14.41% of the dataset.
The distribution of the feature 'Sjögrens_Syndrome' in this dataset is fine.



Linked data saved to ../../output/preprocess/Sjögrens_Syndrome/GSE51092.csv
